In [3]:
data = []
file_name = "questionsData.txt"
with open(file_name) as data_file:
        for l in data_file:
            data.append(l.strip().split(":"))

In [4]:
import random
random.shuffle(data)
print(data[0:10])

[['tell me the median', 'type3'], ['what is the eighth y-axis label', 'type1'], ['tell me the last thirteen x-axis labels', 'type1'], ['which Tahnee is the maximum', 'type3'], ['read me the last six y-axis values', 'type1'], ['what is the Bryanna for proceedings', 'type2'], ['what is the y-axis title', 'type1'], ['what is the kangaroo when act is Brittaney', 'type2'], ['tell me the range value', 'type3'], ['what are the last nineteen x-axis values', 'type1']]


In [5]:
word_embed = {}
import numpy as np
embedding_file = "glove.6B.50d.txt"
count = 0
with open(embedding_file, encoding="utf8") as embeddings:
    for line in embeddings:
        count += 1
        values = line.split()
        vector = np.asarray(values[1:], "float32")
        word_embed[values[0]] = vector

In [6]:
unkStr = '-0.12920076 -0.28866628 -0.01224866 -0.05676644 -0.20210965 -0.08389011 0.33359843  0.16045167  0.03867431  0.17833012  0.04696583 -0.00285802 0.29099807  0.04613704 -0.20923874 -0.06613114 -0.06822549  0.07665912 0.3134014   0.17848536 -0.1225775  -0.09916984 -0.07495987  0.06413227 0.14441176  0.60894334  0.17463093  0.05335403 -0.01273871  0.03474107 -0.8123879  -0.04688699  0.20193407  0.2031118  -0.03935686  0.06967544 -0.01553638 -0.03405238 -0.06528071  0.12250231  0.13991883 -0.17446303 -0.08011883  0.0849521  -0.01041659 -0.13705009  0.20127155  0.10069408 0.00653003  0.01685157'
unkList = unkStr.split()
unkVec = np.asarray(unkList, "float32")
word_embed['<unk>'] = unkVec
word_embed['<pad>'] = np.zeros(50, dtype="float32")
print(word_embed['<pad>'])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


In [7]:
questions = [i[0].split() for i in data]
types = [i[1] for i in data]
for i in range(0, len(questions)):
    for j in range(0, 15-len(questions[i])):
        questions[i].append('<pad>')
print(questions[0:10])
print(types[0:10])
print(questions[len(questions) -1])

[['tell', 'me', 'the', 'median', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'], ['what', 'is', 'the', 'eighth', 'y-axis', 'label', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'], ['tell', 'me', 'the', 'last', 'thirteen', 'x-axis', 'labels', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'], ['which', 'Tahnee', 'is', 'the', 'maximum', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'], ['read', 'me', 'the', 'last', 'six', 'y-axis', 'values', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'], ['what', 'is', 'the', 'Bryanna', 'for', 'proceedings', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'], ['what', 'is', 'the', 'y-axis', 'title', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'], ['what', 'is', 'the', 'kangaroo', 'when', 'act', 'is', 'Brittaney

In [8]:
padData = np.zeros((len(data)*15, 50))
labels = np.zeros(len(data))
label_id = {'type1': 0, 'type2': 1, 'type3': 2}
count = 0
for i in range(0, len(questions)):
    for j in range(0, len(questions[i])):
        if questions[i][j] in word_embed:
            padData[count] = word_embed[questions[i][j]]
        else:
            padData[count] = word_embed['<unk>']
        count += 1
    labels[i] = label_id[types[i]]

In [9]:
print(padData.shape)
print(labels[0:3])

(315000, 50)
[2. 0. 0.]


In [10]:
import torch
import torch.nn as nn
import torch_utils
from torch.autograd import Variable
from torch import optim
import math


class Type_Classifier(nn.Module):
    
    
    def __init__(self, h_size=50, d=50, classes=3, num_layers=2):
        super(Type_Classifier, self).__init__()
        self.hidden_size = h_size
        self.layers = num_layers
        
        # one lSTMs, one for first sentence, one for second
        self.lstm = nn.LSTM(d, h_size, num_layers, batch_first=True)
        
        #after LSTM, output goes through 2 fully connected layers
        self.fc1 = nn.Linear(h_size, 100)
        self.fc2 = nn.Linear(100, classes)
        
    def display(self):
        
        for param in self.parameters():
            print(param.data.size())

    def forward(self, x):
        # takes in 1 input x. x is a 3x15x50 tensor. 
        # Each of the 15 rows is the word embedding for the respective word in the sentence
        
        # x goes through LSTM layer
        h0 = torch.zeros(self.layers, x.size(0), self.hidden_size) 
        c0 = torch.zeros(self.layers, x.size(0), self.hidden_size)
        out, _ = self.lstm(x, (h0,c0))
        
        # concatenate output
        out = out[:, -1, :]
        
        # send through a tanh layer then a linear layer
        outF = self.fc1(out)
        m = nn.Tanh()
        outF = m(outF)
        outF = self.fc2(outF)

        return outF

In [11]:
model = Type_Classifier(h_size=100, d=50, classes=3, num_layers=2)
model.display()

torch.Size([400, 50])
torch.Size([400, 100])
torch.Size([400])
torch.Size([400])
torch.Size([400, 100])
torch.Size([400, 100])
torch.Size([400])
torch.Size([400])
torch.Size([100, 100])
torch.Size([100])
torch.Size([3, 100])
torch.Size([3])


In [12]:
def batch_index_gen(batch_size, size):
    batch_indexer = []
    start = 0
    while start < size:
        end = start + batch_size
        if end > size:
            end = size
        batch_indexer.append((start, end))
        start = end
    return batch_indexer

In [13]:
EPOCHS = 3
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  
for epoch in range(EPOCHS):
    itr = 0
    # using batch size of 3
    for start, end in batch_index_gen(3, len(labels)):

        inp = padData[start*15:end*15]
        lab = labels[start:end]
        
        tensData = torch.FloatTensor(inp)
        tensData = torch.reshape(tensData, [lab.shape[0], 15, 50])
        
        x = tensData
        
        # x is the inputs for the model
        # again x is 3x15x50 sized tensors
        # it is 3 because using batch size 3
        model.zero_grad()
        
        outputs = model(x)
        tensorLab = torch.LongTensor(lab.astype(int))
        tensorLab = torch.reshape(tensorLab, [-1])
#         print(tensorLab)
        loss = criterion(outputs, tensorLab.view(-1))
        
        loss.backward()
        optimizer.step()
        
        if (itr) % 1000 == 0:
            print (f'Epoch [{epoch+1}/{EPOCHS}], Step [{itr+1}/{len(labels)/3}], Loss: {loss.item():.4f}')
        
        itr+=1

Epoch [1/3], Step [1/7000.0], Loss: 1.0854
Epoch [1/3], Step [1001/7000.0], Loss: 0.0133
Epoch [1/3], Step [2001/7000.0], Loss: 0.0010
Epoch [1/3], Step [3001/7000.0], Loss: 0.0004
Epoch [1/3], Step [4001/7000.0], Loss: 0.0002
Epoch [1/3], Step [5001/7000.0], Loss: 0.0002
Epoch [1/3], Step [6001/7000.0], Loss: 0.0001
Epoch [2/3], Step [1/7000.0], Loss: 0.0008
Epoch [2/3], Step [1001/7000.0], Loss: 0.0002
Epoch [2/3], Step [2001/7000.0], Loss: 0.0003
Epoch [2/3], Step [3001/7000.0], Loss: 0.0001
Epoch [2/3], Step [4001/7000.0], Loss: 0.0000
Epoch [2/3], Step [5001/7000.0], Loss: 0.0002
Epoch [2/3], Step [6001/7000.0], Loss: 0.0001
Epoch [3/3], Step [1/7000.0], Loss: 0.0001
Epoch [3/3], Step [1001/7000.0], Loss: 0.0002
Epoch [3/3], Step [2001/7000.0], Loss: 0.0001
Epoch [3/3], Step [3001/7000.0], Loss: 0.0000
Epoch [3/3], Step [4001/7000.0], Loss: 0.0001
Epoch [3/3], Step [5001/7000.0], Loss: 0.0001
Epoch [3/3], Step [6001/7000.0], Loss: 0.0000


In [14]:
testD = []
file_name = "testData.txt"
with open(file_name) as data_file:
        for l in data_file:
            testD.append(l.strip().split(":"))
questions = [i[0].split() for i in testD]
types = [i[1] for i in testD]
for i in range(0, len(questions)):
    for j in range(0, 15-len(questions[i])):
        questions[i].append('<pad>')
padData = np.zeros((len(data)*15, 50))
labels = np.zeros(len(data))
label_id = {'type1': 0, 'type2': 1, 'type3': 2}
count = 0
for i in range(0, len(questions)):
    for j in range(0, len(questions[i])):
        if questions[i][j] in word_embed:
            padData[count] = word_embed[questions[i][j]]
        else:
            padData[count] = word_embed['<unk>']
        count += 1
    labels[i] = label_id[types[i]]

In [16]:
model.eval()

# print(quesData.shape)
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for start, end in batch_index_gen(3, len(labels)):
        inp = padData[start*15:end*15]
        lab = labels[start:end]
        
        tensData = torch.FloatTensor(inp)
        tensData = torch.reshape(tensData, [lab.shape[0], 15, 50])
        
        x = tensData
        
        # x is the inputs for the model
        # again x is 3x15x50 sized tensors
        # it is 3 because using batch size 3
        
        outputs = model(x)
        _, predicted = torch.max(outputs.data, 1)
        
        tensorLab = torch.LongTensor(lab)
        tensorLab = torch.reshape(tensorLab, [3])
        
        n_samples += tensorLab.size(0)
        n_correct += (predicted == tensorLab).sum().item()
        
    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy: {acc} %')

Accuracy: 99.9952380952381 %


In [17]:
import nltk 
# nltk.download()
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize

In [121]:
import pandas as pd
numWrong = 0
dataSets = 100
for i in range(0,dataSets):
    dfName = "data/dataframe" + str(i) + ".csv"
    quesName = "data/ques" + str(i) + ".txt"
    df = pd.read_csv(dfName)
    products_list = [df.columns.values.tolist()][0]
    yLab = products_list[0]
    xLab = products_list[1]
    nums = df[yLab].values.tolist()
    randLabs = df[xLab].values.tolist()
    
    testD = []
    with open(quesName) as data_file:
        for l in data_file:
            testD.append(l.strip().split(":"))
    ques = [i[0] for i in testD]
    ans = [i[1].split(",") for i in testD]
    
    for j in range(0,15):
        prediction = predict(ques[j])
        qList = ques[j].split()
        tags = nltk.pos_tag(qList)
        output = []
        if prediction == 0:
            output = type_0(tags, randLabs, nums, xLab, yLab)
        elif prediction == 1:
            output = type_1(tags, randLabs, nums)
        else:
            output = type_2(tags, randLabs, nums)
        if output != ans[j]:
            numWrong += 1
acc = (((dataSets*15)-numWrong)/(dataSets*15)) * 100
print(acc)

97.13333333333334


In [41]:
def predict(val):
    valList = val.split()
    quesData = np.zeros((15, 50), "float32")
    for i in range(0, len(valList)):
        if valList[i] in word_embed:
            quesData[i] = word_embed[valList[i]]
        else:
            quesData[i] = word_embed['<unk>']
    model.eval()
#     print(quesData.shape)
    with torch.no_grad():
        tens = torch.FloatTensor(quesData)
        tens = torch.reshape(tens, [1, 15, 50])
        outputs = model(tens)

        _, predicted = torch.max(outputs.data, 1)
        prediction = predicted[0].numpy()
        return prediction

In [19]:
import inflect
import random
p = inflect.engine()

In [100]:
### if type 0
wordToInt = {}
for i in range(1,30):
    wordToInt[p.number_to_words(p.ordinal(i))] = i
    wordToInt[p.number_to_words(i)] = i
def type_0(tagged, randLabs, nums, xLab, yLab):
    
    nns = []
    jjs = []
    position = 0
    last = False
    first = False
    ordinal = False
    single = False
    xaxis = True
    title = False

    ind = 0
    for pair in tagged:
        if pair[1] == 'NN':
            nns.append(pair[0])
            if pair[0] == "y-axis":
                xaxis = False
            if pair[0] in wordToInt:
                position = wordToInt[pair[0]]
                if tagged[ind+1][0] == "to" or tagged[ind+1][0] == "from":
                    single = True
            if pair[0] == "last" or pair[0] == "end":
                last = True
        elif pair[1] == 'JJ' or pair[1] == 'CD':
            jjs.append(pair[0])
            if pair[0] == "y-axis":
                xaxis = False
            try:
                position = int(pair[0])
            except:
                pass
            if pair[0] in wordToInt:
                position = wordToInt[pair[0]]
                if tagged[ind+1][0] == "to" or tagged[ind+1][0] == "from":
                    single = True
                try:
                    w2n.word_to_num(pair[0])
                    ordinal = True
                except:
                    pass
            if pair[0] == "first":
                first = True
            if pair[0] == "last" or pair[0] == "end":
                last = True
        ind +=1
    finalp = []
    if first: 
        if position == 0:
            finalp.append(0)
        elif single:
            finalp.append(position-1)
        else:
            for i in range(1, position+1):
                finalp.append(i-1)
    elif last:
        if position == 0:
            finalp.append(-1)
        elif single:
            finalp.append(position*-1)
        else:
            for i in range(position*-1, 0):
                finalp.append(i)
    else:
        if position == 0:
            if xaxis:
                return [xLab]
            else:
                return [yLab]
        finalp.append(position-1)
    retVals = []
    if xaxis:
        for i in range(0, len(finalp)):
            retVals.append(randLabs[finalp[i]])
    else:
        for i in range(0, len(finalp)):
            retVals.append(str(nums[finalp[i]]))
    return retVals

In [118]:
def type_1(tagged, randLabs, nums):
    for pair in tagged:
        if pair[0] in randLabs and (pair[1] == "NN" or pair[1] == "JJ" or pair[1] == "NNP"):
            return [str(nums[randLabs.index(pair[0])])]

In [78]:
from scipy.spatial import distance
import statistics
def type_2(tagged, randLabs, nums):
    operations = ["mean", "average", "median", "mode", "range", "max", "largest", "maximum", "biggest", "tallest", "highest", "min", "smallest", "minimum", "lowest", "shortest"]
    operation = ""
    opN = ""
    opJ = ""
    firstN = True
    for pair in tagged:
        if pair[0] in operations:
            operation = pair[0]
        elif pair[1] == "NN" and firstN:
            opN = pair[0]
            firstN = False
        elif pair[1] == "JJ":
            opJ = pair[0]
    if operation == "":
        minDis = 1
        for op in operations:
            minVal = min(distance.cosine(word_embed[op],word_embed[opJ]), distance.cosine(word_embed[op],word_embed[opN]))
            if minVal < minDis:
                minDis = minVal
                operation = op
    cat = False
    if tagged[0][0] == "which" or tagged[0][0] == "Which":
        cat = True
    opInt = operations.index(operation)
    
    finAns = ''
    if cat:
        if opInt == 2:
            el = statistics.median_high(nums)
            finAns = randLabs[nums.index(el)]
        elif opInt < 11:
            el = max(nums)
            finAns = randLabs[nums.index(el)]
        else:
            el = min(nums)
            finAns = randLabs[nums.index(el)]
    else:
        if opInt < 2:
            finAns = statistics.mean(nums)
        elif opInt < 3:
            finAns = statistics.median(nums)
        elif opInt < 4:
            try:
                finAns = statistics.mode(nums)
            except:
                finAns = "none"
        elif opInt < 5:
            finAns = max(nums) - min(nums)
        elif opInt < 11:
            finAns = max(nums)
        else:
            finAns = min(nums)
    return [str(finAns)]